# HPC Mini-Challenge 2 - Beschleunigung in Data Science
## Teil 2: GPU
#### FHNW - FS2024

Original von S. Suter, angepasst von S. Marcin und M. Stutz

Abgabe von: <font color='blue'>Name hier eintragen</font>

#### Ressourcen
* [Überblick GPU Programmierung](https://www.cherryservers.com/blog/introduction-to-gpu-programming-with-cuda-and-python)
* [CUDA Basic Parts](https://nyu-cds.github.io/python-gpu/02-cuda/)
* [Accelerate Code with CuPy](https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56)
* Vorlesungen und Beispiele aus dem Informatikkurs PAC (parallel computing), siehe Ordner "resources"
* CSCS "High-Performance Computing with Python" Kurs, Tag 3: 
    - JIT Numba GPU 1 + 2
    - https://youtu.be/E4REVbCVxNQ
    - https://github.com/eth-cscs/PythonHPC/tree/master/numba-cuda
    - Siehe auch aktuelles Tutorial von 2021
* [Google CoLab](https://colab.research.google.com/) oder ggf. eigene GPU.


In [1]:
#!pip install numba

DIS ISSUE: https://github.com/numba/numba/issues/7104

NUMBA_CUDA_DRIVER="/usr/lib/wsl/lib/libcuda.so.1" python -c "from numba import cuda; cuda.detect()" -> this works

In [2]:
!numba -s

Der Befehl "numba" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [1]:
# Dummy Beispiel zum testen mit Numba

import math
from numba import vectorize
import numpy as np

@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)
  
a = np.arange(4096,dtype=np.float32)
gpu_sqrt(a)

c:\Users\Stephan\AppData\Local\pypoetry\Cache\virtualenvs\hpc-mc2-buesst1-d94spYeM-py3.11\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 6 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([ 0.       ,  1.       ,  1.4142135, ..., 63.97656  , 63.98437  ,
       63.992188 ], dtype=float32)

### 5 GPU Rekonstruktion

Implementiere eine SVD-Rekonstruktionsvariante auf der GPU oder in einem hybriden Setting. Code aus dem ersten Teil darf dabei verwendet werden. Wähle  bewusst, welche Teile des Algorithms in einem GPU Kernel implementiert werden und welche effizienter auf der CPU sind. Ziehe dafür Erkenntnisse aus dem ersten Teil mit ein. Es muss mindestens eine Komponente des Algorithmuses in einem GPU-Kernel implementiert werden. Dokumentiere Annahmen, welche du ggf. zur Vereinfachung triffst. Evaluiere, ob du mit CuPy oder Numba arbeiten möchtest.

Links:
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)

### GPU-basierte SVD-Rekonstruktion

Die Aufgabe besteht darin, eine Singularwertzerlegung (SVD) effizient auf der GPU zu implementieren, um eine Matrix zu rekonstruieren. Hierbei wurde die Berechnung der einzelnen Elemente der Zielmatrix auf der GPU mittels eines CUDA-Kernels ausgelagert, während Steuerlogik und Speicheroperationen auf der CPU verbleiben. Meine Implementierung basiert vollständig auf Numba.

#### Rekonstruktionsstrategie

Die Rekonstruktion einer Matrix $Y$ aus den SVD-Komponenten $U$, $S$ und $V^T$ basiert auf der Formel:

$$
Y_{m,n} = \sum_{p=1}^{k} U_{m,p} \cdot S_p \cdot V^T_{p,n}
$$

Da $S$ eine Diagonalmatrix ist, kann jeder Eintrag $Y_{m,n}$ unabhängig berechnet werden, indem über die ersten $k$ Singulärwerte iteriert wird. Diese Unabhängigkeit erlaubt eine effiziente Parallelisierung auf der GPU, bei der jedes Element $Y_{m,n}$ gleichzeitig von einem eigenen Thread berechnet wird.

In den ersten Experimenten nutze ich nur das global memory (kein shared). Die nachfolgenden Annahmen treffen aber auch auf den Kernel mit shared memory zu.

#### Designentscheidungen & Annahmen

Eine NVIDIA-GPU besteht aus mehreren **Streaming Multiprocessors (SMs)**, die die parallele Berechnung ermöglichen. Die von mir verwendete NVIDIA RTX 4090 verfügt über 128 SMs. Jeder SM enthält:
- **CUDA-Kerne**, die elementare Operationen ausführen,
- **Register**, die Threads für schnelle Berechnungen (durch schnelleren Memoryzugriff als global Memory) nutzen können,
- **Shared Memory**, der von Threads eines Blocks gemeinsam genutzt werden kann (schnelleres Memory als global Memory, kann von mehreren Threads gemeinsam genutzt werden),
- **L1- und L2-Caches**, die Speicherzugriffe beschleunigen (habe ich als Programmierer keinen direkten Einfluss),
- sowie **Warp-Scheduler**, die Threads in Gruppen, den sogenannten **Warps**, steuern.

Ein Warp besteht aus **32 Threads**, die nach dem **Single Instruction Multiple Data (SIMD)**-Prinzip arbeiten. Das bedeutet, alle Threads eines Warps führen die gleiche Instruktion aus, jedoch auf unterschiedlichen Daten. Divergieren Threads in ihrem Kontrollfluss (z. B. durch if-else-Konstrukte), müssen diese nacheinander ausgeführt werden, was die Effizienz des Warps verringert. Dies wird als **Warp-Divergenz** bezeichnet und sollte bei der Kernel-Implementierung vermieden werden.

Deshalb habe ich meinen Kernel so simpel wie möglich designed, wobei die Warp-Divergenz nur bei Threads auftreten kann, welche sich ausserhalb der Zielmatrix befindet (globale Threadid ausserhalb der Zielmatrix).

#### Blöcke, Threads und Speicherzugriff

Ein **Block** ist eine Sammlung von Threads, die auf einem SM ausgeführt werden. Threads innerhalb eines Blocks können über den **Shared Memory** miteinander kommunizieren, was wesentlich schneller ist als Zugriffe auf den **Global Memory**, der für die gesamte GPU zugänglich ist. Der **Global Memory** hat eine hohe Latenz, daher ist es wichtig, dass Daten, die häufig oder von mehreren Threads benötigt werden, in den Shared Memory verschoben werden. In meinem Fall wäre dies die Diagonalmatrix $S$, welche für jedes Element der Zielmatrix benötigt wird. Im 5.2.2 habe ich diese Verbesserung implementiert.

Die Effizienz des **Global Memory-Zugriffs** hängt von der Anordnung der Daten ab:
- **Memory Coalescing:** Wenn Threads eines Warps auf aufeinanderfolgende Speicheradressen zugreifen, können diese Daten in einem einzigen Ladevorgang übertragen werden.
- **Strided oder random access:** Zugriffe mit unregelmässigen Adressmustern führen zu mehrfachen Speicherzugriffen und reduzieren die Effizienz erheblich.

Die Grösse eines Blocks beeinflusst, wie viele Blöcke gleichzeitig auf einem SM ausgeführt werden können. Zu grosse Blöcke können die Anzahl gleichzeitig laufender Blöcke reduzieren, da die verfügbaren Ressourcen (Register, Shared Memory) eines SMs begrenzt sind. Eine sorgfältige Wahl der Blockgrösse ist daher entscheidend, um die Auslastung der GPU zu maximieren.

Ein paar erste Versuche haben gezeigt, dass die Berechnung der Rekonstruktion am schnellsten ist, wenn alle Inputs in den Kernel row-major sind. Die Ergebnisse habe ich allerdings nirgends aufgezeigt, da mein Code ansonnsten sehr überladen würde. Deshalb sind alle Kernelparameter in meinen Experimenten row-major.

#### Bank Conflicts im Shared Memory

Der **Shared Memory** eines SMs ist in 32 **Speicherbänke** unterteilt, die parallelen Zugriff ermöglichen. Jeder Speicherbank ist ein bestimmter Adressbereich zugeordnet. Ein **Bank Conflict** tritt auf, wenn mehrere Threads eines Warps gleichzeitig auf Adressen zugreifen, die zur gleichen Speicherbank gehören. In diesem Fall müssen die Zugriffe seriell ausgeführt werden, was die Parallelität innerhalb des Warps reduziert.

Bank Conflicts können durch geschickte Datenanordnung und Padding vermieden werden. Dies stellt sicher, dass Threads auf unterschiedliche Speicherbänke zugreifen können, um die maximale Parallelität des Shared Memory zu nutzen.

Um bank-conflicts zu vermeiden habe ich im shared memory kernel die tile-size auf 32 begrenzt.

In [2]:
from src.utils import get_k_timings, make_reconstructor, random_svd, reconstruct_svd_broadcast_timeit
from src.kernels.global_mem import fp32 as kernel_globalmem_fp32
from src.kernels.global_mem import fp64 as kernel_globalmem_fp64

Der Code wurde modularisiert, um die Übersichtlichkeit und Wiederverwendbarkeit zu gewährleisten. Zwei Varianten des CUDA-Kernels wurden implementiert: eine 64-Bit-Version (fp64) für höhere Genauigkeit und eine 32-Bit-Version (fp32) für schnellere Ausführung. Beide Varianten arbeiten ausschliesslich mit globalem Speicher. In einer späteren Version wird der Kernel zusätzlich mit Shared Memory optimiert, um die Effizienz weiter zu steigern.

Die Funktion `make_reconstructor` generiert eine rekonfigurierbare Funktion zur SVD-Rekonstruktion basierend auf dem übergebenen CUDA-Kernel. Sie bietet folgende Parameter:

- **kernel (callable):** Der CUDA-Kernel zur Rekonstruktion.
- **block_size (tuple):** Die Grösse der CUDA-Thread-Blöcke (Threads pro Block).
- **pin_memory (bool):** Option, ob für das Ausgabe-Array gepinnter Speicher verwendet werden soll (beschleunigt den Datenfluss zwischen GPU und Host). Standardmässig deaktiviert.
- **timeit (bool):** Gibt an, ob Laufzeiten gemessen und zurückgegeben werden sollen. Standardmässig deaktiviert.

Basierend auf der Kernel-Signatur werden automatisch die benötigten Datentypen und die Speicheranordnung (Row- oder Column-Major) der Eingabedaten abgeleitet. Dadurch wird sichergestellt, dass die Daten korrekt auf die GPU übertragen und verarbeitet werden. Optional kann auch das Ausgabe-Array in gepinntem Speicher bereitgestellt werden, um den Datentransfer von der GPU zum Host zu beschleunigen.

Die Funktion unterstützt damit eine flexible und effiziente SVD-Rekonstruktion und erlaubt präzise Vergleiche zwischen verschiedenen Konfigurationen, wie z.B. Genauigkeit (fp64) und Geschwindigkeit (fp32).

In [3]:
u, s, vt = random_svd((5000, 5000))
k = len(s)

cpu_reconstruction, time_cpu = reconstruct_svd_broadcast_timeit(u, s, vt, k)

Ich erstelle zuerst zufällige Matrizen, welche sich zu einer 5000x5000 Matrix rekonstruieren lassen. Als k verwende ich die Originalgrösse, damit am meisten Berchnungen durchgeführt werden müssen.

In [4]:
reco_func = make_reconstructor(kernel_globalmem_fp64, (4, 8), timeit=True)

Nun erstelle ich eine Rekonstruktionsfunktion, welche den fp64 Kernel nutzt mit einer Blockgrösse von (4x8). Dieser komplette Block sollte theoretisch auf einem einzelnen Warp ausgeführt werden können, weil 4x8 = 32 = num threads per Warp.

In [5]:
gpu_result, time_gpu = reco_func(u,s,vt,k)

np.allclose(cpu_reconstruction, gpu_result)

True

Wie unschwer zu erkennen ist, hat die Rekonstruktion funktioniert. Die GPU liefert das gleiche Resultat wie wie CPU.

In [6]:
print("CPU Time:")
display(time_cpu)

print("GPU Time:")
display(time_gpu)

CPU Time:


{'total': 0.44700121879577637}

GPU Time:


{'h2d': 0.2714071044921875,
 'd_maloc_y': 2.0479999948292972e-06,
 'h_maloc_y': 0.03838899230957031,
 'kernel': 0.4050489196777344,
 'd2h': 0.03416985702514649,
 'mem_operations_total': 0.34396800182689913,
 'total': 0.7490169215046335}

Hier sind nun verschiedene Zeitmessungen zu erkennen.

Bei der CPU gibt es nur die totale Rekonstruktionszeit.

Bei der GPU wurden noch ein paar weitere Metriken, wie:

- *h2d*: Zeit in Sekunden, welche benötigt wird die Daten von der CPU zur GPU zu senden
- *d_maloc_y*: Zeit in Sekunden, welche benötigt wird die output Matrix auf der GPU zu alloziieren
- *h_maloc_y*: Zeit in Sekunden, welche benötigt wird die output Matrix auf der CPU zu alloziieren
- *kernel*: Kernel execution time in Sekunden
- *d2h*: Zeit in Sekunden welche benötigt wird die Daten von der GPU auf die CPU zu senden
- *mem_operations_total*: h2d + d_maloc_y + h_maloc_y + d2h
- *total*: Totale Zeit in Sekunden

Wie man erkennen kann, benötigt die GPU total mehr Zeit als die CPU. Die Kernel execution time ist allerdings etwas kleiner als die totale Zeit der Berechnung der CPU. Das bedeutet also, dass die Berechnung auf der GPU schneller läuft, allerdings benötigt die Übertragung der Daten zur GPU auch einiges an Zeit, was schlussendlich zu einer langsameren totalen Zeit führt.

In [7]:
reco_func = make_reconstructor(kernel_globalmem_fp32, (4, 8), timeit=True)
gpu_result, time_gpu = reco_func(u,s,vt,k)

np.allclose(cpu_reconstruction, gpu_result)

False

Diesmal habe ich den 32-Bit-Kernel verwendet, da dieser theoretisch deutlich schneller sein sollte als die 64-Bit-Version. GPUs sind bei 32-Bit-Operationen erheblich effizienter. Laut NVIDIA liegt der Durchsatz bei der RTX 4090 bei 64-Bit-Operationen um das 64-Fache unter dem von 32-Bit-Operationen.

Was hier auch deutlich wird ist die Präzision. np.allclose gibt nun ein False zurück. Dies deutet darauf hin, dass die Rekonstruktion nicht mehr die Genauigkeit besitzt wie die 64 Bit Rekonstruktion.

In [8]:
cpu_reconstruction[:4, :4]

array([[  2.36204093,   5.86841491,  58.7303796 ,   7.57380701],
       [ 54.489942  ,  50.75238541, -19.85738721, -18.86543487],
       [ 57.28170276, -84.33807847, -18.3511049 ,  36.32398541],
       [ -0.18313089,  12.39825189, -10.99103981,  44.52368583]])

In [9]:
gpu_result[:4, :4]

array([[  2.3620524 ,   5.868419  ,  58.73034   ,   7.5737495 ],
       [ 54.48995   ,  50.752403  , -19.857382  , -18.865438  ],
       [ 57.281647  , -84.33796   , -18.351053  ,  36.324055  ],
       [ -0.18311755,  12.398203  , -10.991007  ,  44.52362   ]],
      dtype=float32)

Vergleicht man die ersten 16 Elemente der Rekonstruktion, so kann man sehen, dass die Rektionstruktion auf ca 3 Stellen nach dem Komma genau ist.

In [10]:
print("CPU Time:")
display(time_cpu)

print("GPU Time:")
display(time_gpu)

CPU Time:


{'total': 0.44700121879577637}

GPU Time:


{'h2d': 0.13591961669921876,
 'd_maloc_y': 4.0959999896585945e-06,
 'h_maloc_y': 0.06997401428222656,
 'kernel': 0.3099627380371094,
 'd2h': 0.017266687393188478,
 'mem_operations_total': 0.22316441437462348,
 'total': 0.5331271524117329}

Hier zeigt sich, dass sowohl die Kernel-Execution-Time als auch die Transferzeiten von Host zu Device (h2d) und Device zu Host (d2h) bei der 32-Bit-Rekonstruktion kürzer sind als bei der 64-Bit-Version. Dies ist auf zwei Faktoren zurückzuführen: Zum einen sind die Rechenoperationen bei 32 Bit schneller, und zum anderen wird nur die halbe Datenmenge übertragen, was den Datentransfer beschleunigt.

Die Kernel-Execution-Time ist jedoch bei weitem nicht 64-mal kürzer, was darauf hindeutet, dass die Rekonstruktion primär Memory-Bound ist, also durch den Zugriff auf den Speicher limitiert wird.

#### 5.2 GPU-Kernel Performance

##### 5.3.1 Blocks und Input-Grösse

Links: 
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)
* [NVIDIA Kapitel zu "Strided Access"](https://spaces.technik.fhnw.ch/multimediathek/file/cuda-best-practices-in-c)
* https://developer.nvidia.com/blog/cublas-strided-batched-matrix-multiply/
* https://developer.nvidia.com/blog/how-access-global-memory-efficiently-cuda-c-kernels/

Führe 2-3 Experimente mit unterschiedlichen Blockkonfigurationen und Grösse der Input-Daten durch. Erstelle dafür ein neues Datenset mit beliebig grossen Matrizen, da die GPU besonders geeignet ist um grosse Inputs zu verarbeiten (Verwende diese untschiedlich grossen Matrizen für alle nachfolgenden Vergeliche und Tasks ebenfalls). Messe die Performance des GPU-Kernels mittels geeigneten Funktionen. Welche Blockgrösse in Abhängigkeit mit der Input-Grösse hat sich bei dir basierend auf deinen Experimenten als am erfolgreichsten erwiesen? Welches sind deiner Meinung nach die Gründe dafür? Wie sind die Performance Unterschiede zwischen deiner CPU und GPU Implementierung? Diskutiere deine Analyse (ggf. mit Grafiken).

Matrixgrössen

5000x5000

25000x1000

1000x25000

In [57]:
u, s, vt = random_svd((10000, 10))
k = len(s)

reco_func = make_reconstructor(kernel_globalmem_fp64, (1, 32), timeit=True)
reco_func(u, s, vt, k)[1]

{'h2d': 0.09527085113525391,
 'd_maloc_y': 1.0239999974146486e-06,
 'h_maloc_y': 0.00013312000036239626,
 'kernel': 1.5263999812304975e-05,
 'd2h': 0.00012697599828243256,
 'mem_operations_total': 0.09553197113389615,
 'total': 0.09554723513370846}

In [61]:
u, s, vt = random_svd((10000, 10))
k = len(s)

reco_func = make_reconstructor(kernel_globalmem_fp64, (32, 1), timeit=True)
reco_func(u, s, vt, k)[1]

{'h2d': 0.2670059509277344,
 'd_maloc_y': 2.0479999948292972e-06,
 'h_maloc_y': 0.00017023999989032744,
 'kernel': 1.3183999806642532e-05,
 'd2h': 0.00013027200102806093,
 'mem_operations_total': 0.2673085109286476,
 'total': 0.26732169492845426}

##### 5.2.2 Shared Memory auf der GPU
Optimiere deine Implementierung von oben indem du das shared Memory der GPU verwendest. Führe wieder mehrere Experimente mit unterschiedlicher Datengrösse durch und evaluiere den Speedup gegenüber der CPU Implementierung.

Links:
* [Best Practices Memory Optimizations](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)
* [Examples: Matrix Multiplikation und Shared Memory](https://numba.readthedocs.io/en/latest/cuda/examples.html)

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

Was sind deine Erkenntnisse bzgl. GPU-Memory-Allokation und des Daten-Transferes auf die GPU? Interpretiere deine Resultate.

<font color='blue'>Antwort hier eingeben</font>

##### 5.2.3 Bonus: Weitere Optimierungen
Optimiere deine Implementation von oben weiter. Damit du Erfolg hast, muss der Data-Reuse noch grösser sein.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

#### 5.3 NVIDIA Profiler

Benutze einen Performance Profiler von NVIDIA, um Bottlenecks in deinem Code zu identifizieren bzw. unterschiedliche Implementierungen (Blocks, Memory etc.) zu vergleichen. 

* Siehe Beispiel example_profiling_CUDA.ipynb
* [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) für das Profiling des Codes und die Inspektion der Ergebnisse (neuste Variante)
* [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview)
* [Nvidia Visual Profiler](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#visual)

> Du kannst NVIDIA Nsights Systems und den Nvidia Visual Profiler auf deinem PC installieren und die Leistungsergebnisse aus einer Remote-Instanz visualisieren, auch wenn du keine GPU an/in deinem PC hast. Dafür kannst du die ``*.qdrep`` Datei generieren und danach lokal laden.


Dokumentiere deine Analyse ggf. mit 1-2 Visualisierungen und beschreibe, welche Bottlenecks du gefunden bzw. entschärft hast.

<font color='blue'>Antwort hier eingeben inkl. Bild.</font>

### 6 Beschleunigte Rekonstruktion mehrerer Bilder
#### 6.1 Implementierung
Verwende einige der in bisher gelernten Konzepte, um mehrere Bilder gleichzeitig parallel zu rekonstruieren. Weshalb hast du welche Konzepte für deine Implementierung verwenden? Versuche die GPU konstant auszulasten und so auch die verschiedenen Engines der GPU parallel zu brauchen. Untersuche dies auch für grössere Inputs als die MRI-Bilder.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

<font color='blue'>Antwort hier eingeben</font>

#### 6.2 Analyse
Vergleiche den Speedup für deine parallele Implementierung im Vergleich zur seriellen Rekonstruktion einzelner Bilder. Analysiere und diskutiere in diesem Zusammenhang die Gesetze von Amdahl und Gustafson.

<font color='blue'>Antwort hier eingeben</font>

#### 6.3 Komponentendiagramm

Erstelle das Komponentendiagramm dieser Mini-Challenge für die Rekunstruktion mehrere Bilder mit einer GPU-Implementierung. Erläutere das Komponentendigramm in 3-4 Sätzen.


<font color='blue'>Antwort hier eingeben inkl. Bild(ern).</font>

### 7 Reflexion

Reflektiere die folgenden Themen indem du in 3-5 Sätzen begründest und anhand von Beispielen erklärst.

1: Was sind deiner Meinung nach die 3 wichtigsten Prinzipien bei der Beschleunigung von Code?

<font color='blue'>Antwort hier eingeben</font>

2: Welche Rechenarchitekturen der Flynnschen Taxonomie wurden in dieser Mini-Challenge wie verwendet?

<font color='blue'>Antwort hier eingeben</font>

3: Haben wir es in dieser Mini-Challenge hauptsächlich mit CPU- oder IO-Bound Problemen zu tun? Nenne Beispiele.

<font color='blue'>Antwort hier eingeben</font>

4: Wie könnte diese Anwendung in einem Producer-Consumer Design konzipiert werden?

<font color='blue'>Antwort hier eingeben</font>

5: Was sind die wichtigsten Grundlagen, um mehr Performance auf der GPU in dieser Mini-Challenge zu erreichen?

<font color='blue'>Antwort hier eingeben</font>

6: Reflektiere die Mini-Challenge. Was ist gut gelaufen? Wo gab es Probleme? Wo hast du mehr Zeit als geplant gebraucht? Was hast du dabei gelernt? Was hat dich überrascht? Was hättest du zusätzlich lernen wollen? Würdest du gewisse Fragestellungen anders formulieren? Wenn ja, wie?

<font color='blue'>Antwort hier eingeben</font>